# Import and Configure


In [ ]:
#!pip install isbnlib

In [5]:
import pandas as pd
import isbnlib as ib
from isbnlib import ISBNLibException
from bs4 import BeautifulSoup as soup
import requests
import numpy as np
import json
import textwrap
import urllib.request
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from pandas.io.json import json_normalize

In [6]:
## Connect to our service account
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('./cryptic-woods-276221-799aee4b4e94.json', scope)
gc = gspread.authorize(credentials)

# Data Import

In [7]:
spreadsheet_key = '1pKS-yQT4j_JFrQ0vWbAbfeP-2iYGBTD2CP5lM9Fa-Ro'
book = gc.open_by_key(spreadsheet_key)
worksheet = book.worksheet('ISBN')
# get_all_values gives a list of rows.
allrows = worksheet.get_all_values()

labels=allrows.pop(0)
rowdict={}
rowdictkeys=[]
for i in labels:
  rowdictkeys.append(i)
allrowsT=np.transpose(allrows)
#print(allrowsT)
rowdictvalues=allrowsT
rowdict=dict(zip(rowdictkeys,rowdictvalues))
sdf=pd.DataFrame(rowdict)
sdf.head()

,LV Shelf Left,Bedroom Shelf,Other Shelf
0,9780143064978,9788181473561,23
1,9781447209225,9789352500390,43
2,9788190105972,9788131235324,56
3,9780330302296,9788131244951,77
4,9780143330219,9789997637956,93.5


In [8]:
Isbnlist=sdf['Bedroom Shelf'].tolist()

# Lookup Functions

In [9]:
def getBookValsGoogleBooks(isbn="9788129300379"):

  base_api_link = "https://www.googleapis.com/books/v1/volumes?q=isbn:"

  savelist=["publishedDate","title","authors","pageCount","categories","infoLink"]
  namedik={'Author': 'Swami Ramsukhdas','Categories': 'Conduct of life','ISBN-13': '9788129300379','Published': '2010','Title': 'Be Good and Invaluable Advice','URL': 'http://books.google.com/books?id=zOUmMwEACAAJ&dq=isbn:9788129300379&hl=&source=gbs_api'}

  with urllib.request.urlopen(base_api_link + isbn) as f:
    text = f.read()
    decoded_text = text.decode("utf-8")
    obj = json.loads(decoded_text) 

    if obj["totalItems"] <= 0:
      errdict={}
      for i in namedik.keys():
        errdict[i]=""
      return errdict
        
    
    volume_info = obj["items"][0] 
    for i in savelist:
      if i not in volume_info['volumeInfo'].keys():
        volume_info['volumeInfo'][i]=''

    authors = obj["items"][0]["volumeInfo"]["authors"]

  df=pd.DataFrame.from_dict(volume_info)
  df.drop(columns=['kind','id','etag','selfLink','saleInfo','accessInfo'],inplace=True)
  df=df.transpose()
  labeltodroplist=[]

  authlist=df['authors'][0]
  df['authors'][0]=", ".join(authlist)

  if df['categories'] is not '':
    catlist=df['categories'][0]
    df['categories'][0]=", ".join(catlist)
  
  for i in range(len(df.transpose())):
    if df.columns[i] not in savelist:
      labeltodroplist.append(df.columns[i])
  df.drop(inplace=True,axis='columns',labels=labeltodroplist)
  df.rename(columns={"authors":"Author","categories":"Categories","publishedDate":"Published","title":"Title","infoLink":"URL"},inplace=True)
  #df['ISBN-13']=[(isbn)]
  df.drop(inplace=True,columns=["pageCount"])
  print(df.to_dict(orient='records')[0])
  return(df.to_dict(orient='records')[0])


In [10]:
def getpageISBN(isbn="9788129300379"):

  apikey="http://api.scraperapi.com?api_key=6594089066e419a8ba582146468a34af&url="
  apiurl="http://isbnsearch.org/isbn/"

  #r=requests.get(apiurl+isbn)
  #if r is "<Response [403]>":
  req=requests.get(apikey+apiurl+isbn)
  return(req)

def getbookvals(r):
  page_soup=soup(r.content,"html.parser")
  #print(page_soup.prettify())
  booklist=page_soup.findAll("div",{"class":"bookinfo"})
  if len(booklist)==0:
      return {"ISBN-13":isbn}
  page2=soup(str(booklist[0]),"html.parser")
  title=soup((str(page2.find("h1"))),"html.parser").text
  plist=page2.findAll("p")
  infol=[('Title',title)]
  for i in plist:
    vals=i.text.split(sep=": ",maxsplit=1)
    infol.append((vals[0],vals[1]))
  infodict=dict(infol)
  infodict["URL"]=str("http://isbnsearch.org/isbn/"+infodict["ISBN-13"])
  return(infodict)

# Lookup in Google Books and ISBN Search


In [11]:
infolist=[]
row={}
gRowdict={}
for i in Isbnlist:
  gRowdict=getBookValsGoogleBooks(i)
  gRowdict["ISBN-13"]=i
  infolist.append(gRowdict)

{'Title': "Kelley's Textbook of Rheumatology", 'Author': 'Edward D. Harris', 'Published': '2005', 'Categories': 'Arthritis', 'URL': 'http://books.google.co.in/books?id=K2VqAAAAMAAJ&dq=isbn:9789997637956&hl=&source=gbs_api'}
{'Title': 'Textbook of Systemic Vasculitis', 'Author': 'Aman Sharma', 'Published': '2015-03-10', 'Categories': 'Medical', 'URL': 'http://books.google.co.in/books?id=YHsHCgAAQBAJ&dq=isbn:9789351526520&hl=&source=gbs_api'}
{'Title': "Davidson's Principles and Practice of Medicine", 'Author': 'Brian R. Walker, Nicki R. Colledge, Stuart Ralston, Ian D. Penman', 'Published': '2014', 'Categories': 'Diseases', 'URL': 'http://books.google.co.in/books?id=bI3anQEACAAJ&dq=isbn:9780702050473&hl=&source=gbs_api'}
{'Title': 'Medicine Update & Progress in Medicine 2019 -', 'Author': 'KK Pareek, G Narsimulu', 'Published': '2019-05-31', 'Categories': 'Medical', 'URL': 'http://books.google.co.in/books?id=e3KSDwAAQBAJ&dq=isbn:9789352705221&hl=&source=gbs_api'}
{'Title': 'Medical Manag

In [12]:
for i in range(len(infolist)):
  y=infolist[i]
  if (y["Author"] is "" and y["Title"] is ""):
    #not found in google books.
    #find in the web scraper method
    isbn=Isbnlist[i]
    rst=getpageISBN(isbn)
    l1=getbookvals(rst)
    infolist[i]=l1

In [13]:
df=pd.DataFrame(infolist)
df.drop(inplace=True,columns=["ISBN-10","Edition","Binding","Publisher"])
df.fillna("Not Available")
df.to_csv("./data.csv",index=False)
df

,ISBN-13,Title,Authors,Published,URL,Author,Categories
0,9788181473561,NaN,NaN,NaN,NaN,NaN,NaN
1,9789352500390,CARDIOLOGY UPDATED 2015 (CARDIOLOGICAL SOCIETY...,SANTANU GUHA; SANTANU GUHA; SANTANU GUHA,January 2015,http://isbnsearch.org/isbn/9789352500390,NaN,NaN
2,9788131235324,NaN,NaN,NaN,NaN,NaN,NaN
3,9788131244951,NaN,NaN,NaN,NaN,NaN,NaN
4,9789997637956,Kelley's Textbook of Rheumatology,NaN,2005,http://books.google.co.in/books?id=K2VqAAAAMAA...,Edward D. Harris,Arthritis
5,9789997637949,NaN,NaN,NaN,NaN,NaN,NaN
6,9780071842327,NaN,NaN,NaN,NaN,NaN,NaN
7,9781631025976,NaN,NaN,NaN,NaN,NaN,NaN
8,9780071842303,NaN,NaN,NaN,NaN,NaN,NaN
9,9789351341932,NaN,NaN,NaN,NaN,NaN,NaN


# Lookup on OpenLib

In [14]:
df=pd.read_csv("./data.csv",)
df.fillna("")

,ISBN-13,Title,Authors,Published,URL,Author,Categories
0,9788181473561,,,,,,
1,9789352500390,CARDIOLOGY UPDATED 2015 (CARDIOLOGICAL SOCIETY...,SANTANU GUHA; SANTANU GUHA; SANTANU GUHA,January 2015,http://isbnsearch.org/isbn/9789352500390,,
2,9788131235324,,,,,,
3,9788131244951,,,,,,
4,9789997637956,Kelley's Textbook of Rheumatology,,2005,http://books.google.co.in/books?id=K2VqAAAAMAA...,Edward D. Harris,Arthritis
5,9789997637949,,,,,,
6,9780071842327,,,,,,
7,9781631025976,,,,,,
8,9780071842303,,,,,,
9,9789351341932,,,,,,


In [15]:
nfdict={}
nflist=[]
for i in range(len(df['Title'])):
    if (pd.isna(df['Title'][i])):
        nfdict["index"]=i
        print(i,df["ISBN-13"][i])
        nfdict["isbn"]=df["ISBN-13"][i]
        nflist.append(nfdict)
        nfdict={}

0 9788181473561
2 9788131235324
3 9788131244951
5 9789997637949
6 9780071842327
7 9781631025976
8 9780071842303
9 9789351341932
10 9780199458080
11 9788132203773
12 9789385880926
13 9789352606344
14 9788193534267
15 9788193534274
16 9788126573660
17 9780198809937
19 9788131254417
20 9781943438198
25 9788132244943
28 9788181479136
29 9780199456802
31 9781631026553
34 9789380780375
35 6802073
36 9788184733341
37 9789352501991
39 9789387506145
40 9788131254967
43 9789387963511
45 9780198789246
46 9781856422372
47 9781580406529


In [16]:
nfinfolist=[]
infodict={}

notFound=False
namesofmeta=['Authors','Language','Publisher','Title','Year']


for i in nflist:
    isbnval=list((i.values()))[1]
    print((isbnval))
    infodict={}
    exfodict={}
   
    try:
        infodict= ib.meta(str(isbnval),service="openl")
    except ISBNLibException:
        infodict = {"ISBN-13":str(isbnval)}
        notFound=True
        for i in namesofmeta:
          infodict[i]=""
      
    if not(False):
      authname=""
      for j in infodict["Authors"]:
        if not isinstance(infodict["Authors"],str):
          authname=", ".join(j)
        else:
          authname=str(infodict["Authors"])
      exfodict["Author"]=authname
      exfodict["Title"]=infodict["Title"]
      exfodict["ISBN-13"]=infodict["ISBN-13"]
      exfodict["Published"]=infodict["Year"]
      exfodict["URL"]=""
      exfodict["Categories"]=""
      #exfodict["Publisher"]=infodict["Publisher"]
    print(exfodict)
    
    nfinfolist.append(exfodict)  

9788181473561
{'Author': '', 'Title': '', 'ISBN-13': '9788181473561', 'Published': '', 'URL': '', 'Categories': ''}
9788131235324
{'Author': '', 'Title': '', 'ISBN-13': '9788131235324', 'Published': '', 'URL': '', 'Categories': ''}
9788131244951
{'Author': '', 'Title': '', 'ISBN-13': '9788131244951', 'Published': '', 'URL': '', 'Categories': ''}
9789997637949
{'Author': '', 'Title': '', 'ISBN-13': '9789997637949', 'Published': '', 'URL': '', 'Categories': ''}
9780071842327
{'Author': '', 'Title': '', 'ISBN-13': '9780071842327', 'Published': '', 'URL': '', 'Categories': ''}
9781631025976
{'Author': '', 'Title': '', 'ISBN-13': '9781631025976', 'Published': '', 'URL': '', 'Categories': ''}
9780071842303
{'Author': '', 'Title': '', 'ISBN-13': '9780071842303', 'Published': '', 'URL': '', 'Categories': ''}
9789351341932
{'Author': '', 'Title': '', 'ISBN-13': '9789351341932', 'Published': '', 'URL': '', 'Categories': ''}
9780199458080
{'Author': '', 'Title': '', 'ISBN-13': '9780199458080', 'P

In [18]:
nfupdatelist=[]
for i in nfinfolist:
  row=[]
  row.append(i["Title"])
  row.append(i["Author"])
  row.append(i["Published"])
  row.append(i["Categories"])
  row.append(i["URL"])
  row.append(i["ISBN-13"])
  nfupdatelist.append(row)

j=0
for i in nflist:
  index=list(i.values())[0]
  df.loc[index]=nfupdatelist[j]
  j=j+1

df

ValueError: Must have equal len keys and value when setting with an iterable

# Lookup on Amazon and others

In [19]:
def scrapeamazon(isbn="9788181479136"):
  apihead="http://api.scraperapi.com?api_key=6594089066e419a8ba582146468a34af&url="
  pageurl="https://www.amazon.com/s?k="
  r=requests.get(pageurl+isbn)
  #if (r.status_code>500):
  #  r1=requests.get(apihead+pageurl+isbn)
  #  r=r1
  page_soup=soup(r.content,"html.parser")
  datalist=page_soup.findAll("div",{"class":"sg-col-inner"})
  return(datalist)

#scrapeamazon()

In [20]:
def scrapeisbnsnet(isbn="9788189988364"):
  apihead="http://api.scraperapi.com?api_key=6594089066e419a8ba582146468a34af&url="
  pageurl="https://www.isbns.net/isbn/"
  r=requests.get(apihead+pageurl+isbn+"/")
  print(r.content)
  page_soup=soup(r.content,"html.parser")
  datalist=page_soup.findAll("td",{"class":"regularText"})
  return(datalist)

#datalist=scrapeisbnsnet()

In [21]:
df.to_csv("./output.csv",index=False)

In [22]:
from df2gspread import df2gspread as d2g

wks_name = 'Jupyter Manipulated Data'

d2g.upload(df, spreadsheet_key, wks_name, credentials=credentials, row_names=True)

<Worksheet 'Jupyter Manipulated Data' id:1253292659>